# Reading in MCTS (Monte Carlo Tree Search) data
### The goal is to train two RNNs.
 - Predict Probability of each of the moves being successful
 - Evaluate the value of the current board position

In [1]:
# uninstall numpy and then reinstall/update tensorflow
import pandas as pd
import numpy as np
print(np.__version__) # must not be the most recent. 1.26.4 works.
import tensorflow as tf
print(tf.__version__)
  
from board import Board
import math
import pickle
import gzip

1.26.4
2.17.0


In [19]:
def loadall(filename, n=math.inf):
    data = []
    with gzip.open(filename, "rb", compresslevel=9) as f:
        while len(data) < n:
            try:
                df = pickle.load(f)
                data.append(df)
            except EOFError:
                break
            
            print(len(data))
            
    return data

In [29]:
import pandas as pd 
print(pd.__version__) # should work with 2.2.2
from board import Board

df_total = pd.DataFrame()

items = loadall('Data/playing_data_saved.pkl')
for value in items:
    df_total = pd.concat([df_total, value])
print(df_total.shape, ' shape')

# items = loadall('Data/playing_data_saved.pkl')
# for value in items:
#     df_total = pd.concat([df_total, value])    
# print(df_total.shape, ' shape')

# items = loadall('Data/playing_data_saved1.pkl')
# for value in items:
#     df_total = pd.concat([df_total, value])
# print(df_total.shape, ' shape')

# items = loadall('Data/playing_data.pkl')
# for value in items:
#     df_total = pd.concat([df_total, value])


print(df_total.shape, ' shape')
print(df_total.size, ' size')

2.2.2
1
(165335, 8)  shape
(165335, 8)  shape
1322680  size


In [30]:
print(df_total.columns)

Index(['Board', 'Moves', 'Move_Probs', 'Reward', 'Weights', 'Num_Sims',
       'mc_type', 'inv_left'],
      dtype='object')


In [26]:
# checking out to make the data is stored correctly
  
for i in range(20):
    print(i, df_total.iloc[i]['Move_Probs'])
    # x = df_total.iloc[i]['Board']
    # board = Board(14)
    # board.show_dots = False
    # board.board = x
    # print(board)



0 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [34]:
import copy

all_poss_moves = []
for x in range(14):
  for y in range(14):
     for piece in range(21):
        for orient in range(8):
           all_poss_moves.append([x,y,piece, orient])
         
changed_df = df_total.copy()
print('cloneing done', changed_df.head)

for index, row in df_total.iterrows():
    print(index)
    moves_probs = []
    hist_moves = row['Moves']
    hist_moves_probs = row['Move_Probs']
    for poss_move in all_poss_moves:
        if poss_move in hist_moves:
            index = hist_moves.index(poss_move)
            moves_probs.append(hist_moves_probs[index])
        else:
            moves_probs.append(0)
    changed_df.iloc[index]['Move_Probs'] = moves_probs

cloneing done <bound method NDFrame.head of                                                 Board  \
0   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
1   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
2   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
3   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
4   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
..                                                ...   
55  [[0, 0, 0, 0, 1, 1, 0, 0, 1, 0, -1, -1, -1, -1...   
56  [[0, 0, 0, 0, 1, 1, 0, 0, 1, 0, -1, -1, -1, -1...   
57  [[-1, 0, 0, 0, 1, 1, 0, 0, 1, 0, -1, -1, -1, -...   
58  [[-1, 0, 0, 0, 1, 1, 0, 0, 1, 0, -1, -1, -1, -...   
59  [[-1, 0, 0, 0, 1, 1, 0, 0, 1, 0, -1, -1, -1, -...   

                                                Moves  \
0   [[3, 5, 9, 1, 0, 1], [4, 4, 20, 0, 0, 0], [3, ...   
1   [[10, 8, 11, 7, 0, 1], [9, 11, 7, 1, 0, 0], [1...   
2   [[6, 3, 6, 1, 0, 2], [5, 6, 0, 0, 1, 0], [5, 8...   
3   [[11, 5, 7, 0, 0, 2], [10, 10, 20, 4, 2

C:\Users\jjfri\AppData\Local\Temp\ipykernel_23068\3146737014.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  changed_df.iloc[index]['Move_Probs'] = moves_probs


4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14

In [9]:
x = changed_df.iloc[28]['Move_Probs']
print(x)

32928 4


# Working on the DCNNs
### References:
 - https://medium.com/aiguys/deep-convolutional-neural-networks-dcnns-explained-in-layman-terms-b990b2818061
  - https://nikcheerla.github.io/deeplearningschool/2018/01/01/AlphaZero-Explained/


In [10]:
def preprocess_data(games, train_test_ratio=.8):
    # Convert game records into input-output pairs
    games = games.sample(frac = 1)
    split_index = math.floor(train_test_ratio*(games.shape[0]))
    sameation_num = 10
    print("Amount of training batches: ", split_index)
    train_df = games.iloc[:split_index-sameation_num]
    test_df = games.iloc[split_index:]
    sameation_df = games.iloc[split_index-sameation_num:split_index]

    train_boards = np.array(tuple(train_df['Board'].values))
    train_rewards = np.array(tuple(train_df['Reward'].values))
    train_moves = np.array(tuple(train_df['Reward'].values))

    test_boards = np.array(tuple(test_df['Board'].values))
    test_rewards = np.array(tuple(test_df['Reward'].values))
    
    x_val = np.array(tuple(sameation_df['Board'].values))
    y_val = np.array(tuple(sameation_df['Reward'].values))
    
    train_boards = np.reshape(train_boards, (train_boards.shape[0], train_boards.shape[1], train_boards.shape[2], 1))

    
    processed_data = {
        "train boards": train_boards,
        "train rewards": train_rewards,
        "test boards": test_boards,
        "test rewards": test_rewards,
        "x val": x_val,
        "y val": y_val
    }
    
    # Normalize and format the data for training
    return processed_data

In [11]:
# Defining the evaluation snd predictive networks
def create_policy_network(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(14*14*21*8, activation='softmax')
    ])
    return model

def create_value_network(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=6, activation='relu', input_shape=input_shape, padding='same'),
        tf.keras.layers.BatchNormalization(), 
        tf.keras.layers.Conv2D(128, kernel_size=6, activation='relu', padding='valid'),
        tf.keras.layers.BatchNormalization(), 
        tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='valid'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4096, input_shape = (14*14*1, )),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(4096),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(2048),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, activation='tanh')
    ])
    return model


def create_value_network1(input_shape):
    model = tf.keras.Sequential([

    tf.keras.layers.Conv2D(filters = 96, input_shape = input_shape,  
                kernel_size = (5, 5),  strides = (4,4),
                padding = 'same'),
    tf.keras.layers.Activation('relu'),
    # Max-Pooling  
    tf.keras.layers.MaxPooling2D(pool_size = (2, 2), 
                strides = (2, 2), padding = 'same'),
    # # Batch Normalisation 
    tf.keras.layers.BatchNormalization(), 
    
    # 2nd Convolutional Layer 
    tf.keras.layers.Conv2D(filters = 30, kernel_size = (6, 6),  
                strides = (1, 1), padding = 'same'), 
    tf.keras.layers.Activation('relu'), 
    # Max-Pooling 
    tf.keras.layers.MaxPooling2D(pool_size = (2, 2),  
                padding = 'same'),
    # Batch Normalisation 
    tf.keras.layers.BatchNormalization(),
    
    # 3rd Convolutional Layer 
    tf.keras.layers.Conv2D(filters = 384, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'same'),
    tf.keras.layers.Activation('relu'),
    # Batch Normalisation 
    tf.keras.layers.BatchNormalization(),
    
    # # 4th Convolutional Layer 
    # tf.keras.layers.Conv2D(filters = 384, kernel_size = (3, 3),  
    #             strides = (1, 1), padding = 'same'),
    # tf.keras.layers.Activation('relu'),
    # # Batch Normalisation 
    # tf.keras.layers.BatchNormalization(),
    
    # # 5th Convolutional Layer 
    # tf.keras.layers.Conv2D(filters = 256, kernel_size = (3, 3),  
    #             strides = (1, 1), padding = 'same'),
    # tf.keras.layers.Activation('relu'),
    # # Max-Pooling 
    # tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2),  
    #             padding = 'same'),
    # # Batch Normalisation 
    # tf.keras.layers.BatchNormalization(),
    
    # Flattening 
    tf.keras.layers.Flatten(),
    
    # 1st Dense Layer 
    tf.keras.layers.Dense(4096, input_shape = (14*14*1, )),
    tf.keras.layers.Activation('relu'),
    # Add Dropout to prevent overfitting 
    tf.keras.layers.Dropout(0.5),
    # Batch Normalisation 
    tf.keras.layers.BatchNormalization(),
    
    # 2nd Dense Layer 
    tf.keras.layers.Dense(4096),
    tf.keras.layers.Activation('relu'),
    # Add Dropout 
    tf.keras.layers.Dropout(0.5),
    # Batch Normalisation 
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Dense(1, activation='tanh')

    ])
    return model

def create_value_network_AlexNet(input_shape):
    model = tf.keras.Sequential() 
    # 1st Convolutional Layer 
    model.add(tf.keras.layers.Conv2D(filters = 32, input_shape = input_shape,  
                kernel_size = (6, 6), strides = (2, 2),  
                padding = 'valid')) 
    model.add(tf.keras.layers.Activation('relu')) 
    # Max-Pooling  
    # model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2), 
    #             strides = (2, 2), padding = 'valid')) 
    # Batch Normalisation 
    model.add(tf.keras.layers.BatchNormalization()) 
    
    # 2nd Convolutional Layer 
    model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (4, 4),  
                strides = (1, 1), padding = 'valid')) 
    model.add(tf.keras.layers.Activation('relu')) 
    # Max-Pooling 
    # model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2),  
    #             padding = 'valid')) 
    # Batch Normalisation 
    model.add(tf.keras.layers.BatchNormalization()) 
    
    # 3rd Convolutional Layer 
    model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'same')) 
    model.add(tf.keras.layers.Activation('relu')) 
    # Batch Normalisation 
    model.add(tf.keras.layers.BatchNormalization()) 
    
    # 4th Convolutional Layer 
    model.add(tf.keras.layers.Conv2D(filters = 84, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'same')) 
    model.add(tf.keras.layers.Activation('relu')) 
    # Batch Normalisation 
    model.add(tf.keras.layers.BatchNormalization()) 
    
    # 5th Convolutional Layer 
    model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'same')) 
    model.add(tf.keras.layers.Activation('relu')) 
    # Max-Pooling 
    model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2),  
                padding = 'valid')) 
    # Batch Normalisation 
    model.add(tf.keras.layers.BatchNormalization()) 
    
    # Flattening 
    model.add(tf.keras.layers.Flatten()) 
    
    # 1st Dense Layer 
    model.add(tf.keras.layers.Dense(4096, input_shape = (14*14*1, ))) 
    model.add(tf.keras.layers.Activation('relu')) 
    # Add Dropout to prevent overfitting 
    model.add(tf.keras.layers.Dropout(0.4)) 
    # Batch Normalisation 
    model.add(tf.keras.layers.BatchNormalization()) 
    
    # 2nd Dense Layer 
    model.add(tf.keras.layers.Dense(4096)) 
    model.add(tf.keras.layers.Activation('relu')) 
    # Add Dropout 
    model.add(tf.keras.layers.Dropout(0.4)) 
    # Batch Normalisation 
    model.add(tf.keras.layers.BatchNormalization()) 
    
    # Output Softmax Layer 
    model.add(tf.keras.layers.Dense(1)) 
    model.add(tf.keras.layers.Activation('sigmoid'))
    
    return model 

In [12]:
# Code snippet for training the neural networks
def train_networks(policy_network, value_network, data, epochs=10, batch_size=32):
    # Define loss functions and optimizer
    policy_loss = tf.keras.losses.CategoricalCrossentropy()
    value_loss = tf.keras.losses.MeanSquaredError()
    optimizer = tf.keras.optimizers.Adam()
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

    # Compile the models
    policy_network.compile(optimizer=optimizer, loss=policy_loss, metrics=metrics)
    value_network.compile(optimizer=optimizer, loss=value_loss, metrics=metrics)

    # Train the models
    # policy_network.fit(processed_data['input'], processed_data['policy_target'], epochs=epochs, batch_size=batch_size, sameation_data=(x_val, y_val))
    value_network.fit(data['train boards'], data['train rewards'], epochs=epochs, batch_size=batch_size, validation_data=(data['x val'], data['y val']))

In [13]:
board_size = 14
print(np.shape(df_total))
data = preprocess_data(df_total, train_test_ratio=.8)
print(np.shape(data["train boards"])[0])

input_shape = (board_size, board_size, 1)

policy_network = create_policy_network(input_shape)
value_network = create_value_network(input_shape)
# value_network = create_value_network_AlexNet(input_shape)

train_networks(policy_network, value_network, data, epochs=10, batch_size=32)

(165335, 8)
Amount of training batches:  132268
132258


C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
4134/4134 ━━━━━━━━━━━━━━━━━━━━ 594s 142ms/step - loss: 1.9164 - sparse_categorical_accuracy: 0.0236 - val_loss: 0.9021 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/10
4134/4134 ━━━━━━━━━━━━━━━━━━━━ 587s 142ms/step - loss: 1.2647 - sparse_categorical_accuracy: 0.0242 - val_loss: 0.8611 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/10
4134/4134 ━━━━━━━━━━━━━━━━━━━━ 584s 141ms/step - loss: 1.1331 - sparse_categorical_accuracy: 0.0235 - val_loss: 0.9033 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 4/10
4134/4134 ━━━━━━━━━━━━━━━━━━━━ 1593s 386ms/step - loss: 1.1026 - sparse_categorical_accuracy: 0.0236 - val_loss: 1.0750 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 5/10
4134/4134 ━━━━━━━━━━━━━━━━━━━━ 548s 132ms/step - loss: 1.0756 - sparse_categorical_accuracy: 0.0231 - val_loss: 0.9868 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 6/10
4134/4134 ━━━━━━━━━━━━━━━━━━━━ 453s 110ms/step - loss: 1.0474 - sparse_categorical_accuracy: 0.0236 - val_lo

In [14]:
to_pred = np.array((tuple(df_total.iloc[14]['Board'])))
to_pred = to_pred[np.newaxis, :, :, np.newaxis]
print(np.shape(to_pred), df_total.iloc[14]['Reward'])
value_network.predict(to_pred)

(1, 14, 14, 1) -1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step


array([[0.09975834]], dtype=float32)

In [15]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = value_network.evaluate(data['test boards'], data['test rewards'], batch_size=12)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
for i in range(20):
    # print("Generate predictions for 3 samples")
    to_pred = np.array(tuple(data['test boards'][i]))
    print(to_pred)
    print(data['test rewards'][i])
    to_pred = to_pred[np.newaxis, :, :, np.newaxis]

    predictions = value_network.predict(to_pred)
    print("predictions shape:", predictions.shape)
    print("prediction:", predictions, "correct reward: ", data['test rewards'][i])

Evaluate on test data
2756/2756 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 1.0105 - sparse_categorical_accuracy: 0.0234
test loss, test acc: [1.0131152868270874, 0.0226812232285738]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  1  0  0  1  1  0  0  0  0  0  0  0]
 [ 0  0  1  1  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 -1 -1 -1  0  0  0]
 [ 0  0  0  0  0  0 -1 -1  0 -1  0 -1  0  0]
 [ 0  0  0  0  0 -1 -1  0  0  0  0 -1 -1 -1]
 [ 0  0  0  0  0  0 -1  0  0  0  0 -1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
predictions shape: (1, 1)
prediction: [[0.06293959]] correct reward:  1
[[ 0  0 -1 -1 -1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 -1 -1  0  0 -1  0  0

In [16]:
import tensorflow as tf

value_net_path = "models/value_net_basic.keras"
value_network.save(value_net_path)

In [17]:
value_network_saved = tf.keras.models.load_model(value_net_path)
value_network_saved.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 14, 14, 32)     │         1,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 14, 14, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 9, 9, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 9, 9, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 7, 7, 64)       │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4096)           │    12,849,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 4096)           │        16,384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 4096)           │        16,384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 2048)           │     8,390,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │         2,049 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 114,820,389 (438.01 MB)

 Trainable params: 38,266,529 (145.98 MB)

 Non-trainable params: 20,800 (81.25 KB)

 Optimizer params: 76,533,060 (291.95 MB)